# Preparing the Dataset

In [1]:
import numpy as np
import pandas as pd
import os
import pymongo

In [2]:
npy_file = os.path.join("data","Fire_Data.npy")

In [3]:
pickle = np.load(npy_file, allow_pickle=True)

In [4]:
fires_df = pd.DataFrame(pickle, columns=['FPA_ID','FIRE_NAME','FIRE_YEAR','DISCOVERY_DOY','DISCOVERY_TIME','CONT_DOY','CONT_TIME','FIRE_SIZE','FIRE_SIZE_CLASS','LATITUDE','LONGITUDE','STATE','COUNTY','FIPS_CODE','FIPS_NAME','STAT_CAUSE_DESCR','OWNER_CODE','OWNER_DESCR','DAYS_TO_CONT'])

In [5]:
fires_df.head()

,FPA_ID,FIRE_NAME,FIRE_YEAR,DISCOVERY_DOY,DISCOVERY_TIME,CONT_DOY,CONT_TIME,FIRE_SIZE,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,STATE,COUNTY,FIPS_CODE,FIPS_NAME,STAT_CAUSE_DESCR,OWNER_CODE,OWNER_DESCR,DAYS_TO_CONT
0,FS-1418826,FOUNTAIN,2005,33,1300,33,1730,0.1,A,40.0369,-121.006,CA,63,063,Plumas,Miscellaneous,5,USFS,0
1,FS-1418827,PIGEON,2004,133,0845,133,1530,0.25,A,38.9331,-120.404,CA,61,061,Placer,Lightning,5,USFS,0
2,FS-1418835,SLACK,2004,152,1921,152,2024,0.1,A,38.9842,-120.736,CA,17,017,El Dorado,Debris Burning,13,STATE OR PRIVATE,0
3,FS-1418845,DEER,2004,180,1600,185,1400,0.1,A,38.5592,-119.913,CA,3,003,Alpine,Lightning,5,USFS,5
4,FS-1418847,STEVENOT,2004,180,1600,185,1200,0.1,A,38.5592,-119.933,CA,3,003,Alpine,Lightning,5,USFS,5


In [6]:
## Setup Mongo DB
client = pymongo.MongoClient()

In [7]:
## Build DataBase
db = client.Project_2_db
fires = db.fires

In [8]:
fires.insert_many(fires_df.to_dict('records'))